In [11]:
import sys; sys.path.insert(0,'..')
from ppanlp import *
# !python -m spacy download en_core_web_lg
from tqdm import tqdm
import random
import spacy

In [12]:
model = spacy.load("en_core_web_lg")

In [53]:

def find_people(filename1, shuffle=False, lim=False):
    filename=os.path.join(PATH_TEXT_CORPUS_TEXTS, filename1)
    if os.path.exists(filename):
        with open(filename) as f: txt=f.read()
        txt=txt.replace('-\n','').replace('- ','').replace('ſ','s')
        pages = txt.strip().split('\n\n\n')
        pages = list(enumerate(pages))
        if shuffle: random.shuffle(pages)
        pages=pages[:lim]
        for page_i,page in pages:
            nerpage = model(page)
            for ent in nerpage.ents:
                yield dict(filename=filename1,page_i=page_i,token=ent.text,enttype=ent.label_)


def find_all_people(fn='data.PPA.people.v2.jsonl'):
    df=get_txtcorpus_metadata()
    filenames=df.filename
    with jsonlines.open(fn,'w') as of:
        for filename in tqdm(filenames):
            for d in find_people(filename,shuffle=False,lim=None):
                of.write(d)


In [43]:
find_all_people()

In [45]:
df=pd.read_json('./data.PPA.people.jsonl', lines=True)
df

,filename,page_i,token,enttype
0,CB0126124028.txt,1,one,CARDINAL
1,CB0126124028.txt,1,flill,PERSON
2,CB0126124028.txt,1,one,CARDINAL
3,CB0126124028.txt,1,WVork,ORG
4,CB0126124028.txt,1,Wrriters,WORK_OF_ART
...,...,...,...,...
147702,yale.39002088447587.txt,267,dclayest,ORG
147703,yale.39002088447587.txt,267,trs,NORP
147704,yale.39002088447587.txt,267,1,CARDINAL
147705,yale.39002088447587.txt,267,2,CARDINAL


In [51]:
df[df.enttype.isin({'PERSON','ORG'})].token.value_counts().head(25)

token
Shakespeare    264
Milton         207
Chaucer        205
John           193
un             174
adv            115
n.             112
Johnson        106
Spenser        103
Tennyson       101
Mary            93
Pope            93
James           90
William         87
EXERCISES       84
Dryden          82
Shakspeare      81
Cicero          77
MS              75
Coleridge       74
Browning        73
Shelley         72
Bacon           67
Scott           67
Dante           66
Name: count, dtype: int64